In [1]:
# The basics
import numpy as np
import pandas as pd

# Import Sklearn model libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score, precision_recall_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA


# Import Utility libraries
import shap
from collections import Counter
import pickle
from tqdm import tqdm_notebook
import warnings
import ipyparallel as ipp
import time
from termcolor import colored
import os
# Attack Utilities
# from create_models import create_models
# from attack import attack
# from attack_utils import baseline, load_data
# from attack_data import attack_dataset, run_engines

# Pre-run necessities
warnings.filterwarnings('ignore')
rnds = [60, 452, 774, 802, 961, 626, 726, 211, 375, 448, 883, 684, 724, 333, 64, 646, 116, 714, 483, 73, 562, 703, 276, 394, 190, 675, 314, 604, 297, 266, 456, 845, 822, 529, 420, 605, 935, 733, 167, 603, 401, 948, 241, 734, 550, 65, 429, 470, 633, 627, 223, 713, 958, 40, 200, 641, 357, 778, 781, 498, 202, 349, 983, 125, 548, 331, 206, 556, 356, 805, 246, 626, 358, 393, 307, 792, 777, 169, 595, 279, 719, 902, 124, 197, 983, 499, 368, 864, 896, 887, 879, 224, 220, 926, 565, 173, 919, 3, 908, 941]

In [2]:
class create_models():
    
    '''
    Create datasets and models on the given dataset
    '''
    
    def __init__(self,x,y,target_size,n_models,random_state,want_tar_tuning,want_ref_tuning,tune_scoring,params,know,n_est,max_d):
        
        '''
        Input should be of the size (NxD), Labels should be of the size (Nx1)
        test_size_target,test_size_ref
        '''
        self.x = x
        self.y = y
        self.target_size = target_size
        self.random_state = random_state
        self.n_models = n_models
        self.ref_model = []
        self.tune_scoring = tune_scoring
        self.n_est = n_est
        self.max_d = max_d
#         if know == 'disjoint':
#             self.create_datasets_disjoint()
#         elif know == 'shared':
#             self.create_datasets_shared()
        self.create_datasets_disjoint()
        self.train_target(want_tar_tuning,params)
        self.train_reference(want_ref_tuning,params)
        tr_acc,te_acc = self.target_performance()
        
#         print(colored("Target Metrics",'green'),end=": ")
#         print("Train Accuracy= {a}, Test Accuracy= {b}".format(a=np.round(tr_acc,3),b=np.round(te_acc,3)))
        
    def set_parameters(self,model,params,x,y):
        
        grid_search = GridSearchCV(model,params,scoring=self.tune_scoring,n_jobs=-1)
        grid_search.fit(x,y)
        best_parameters = grid_search.best_params_
        return best_parameters
        
    def create_datasets_disjoint(self):
        
        # Make target and reference pool distinct
        x_ref,x_tar,y_ref,y_tar = train_test_split(self.x,self.y,test_size=self.target_size,random_state=self.random_state)
        self.target_data_full = (x_tar,y_tar) # for Yeom et al approach
        self.ref_data_full = (x_ref,y_ref) # for Yeom et al approach
        
        # Split the target data into train and test
        self.x_tar_train,self.x_tar_test,self.y_tar_train,self.y_tar_test = train_test_split(x_tar,y_tar,test_size=0.5,random_state=self.random_state)
        x_ref_train_,x_ref_test_,y_ref_train_,y_ref_test_ = [],[],[],[]
        ref_rnds = [60, 452, 774, 802, 961, 626, 726, 211, 375, 448, 883, 684, 724, 333, 64, 646, 116, 714, 483, 73, 562, 703, 276, 394, 190, 675, 314, 604, 297, 266, 456, 845, 822, 529, 420, 605, 935, 733, 167, 603, 401, 948, 241, 734, 550, 65, 429, 470, 633, 627]
        for i in range(self.n_models):
            x_ref_train,x_ref_test,y_ref_train,y_ref_test = train_test_split(x_ref,y_ref,test_size=0.5,random_state=ref_rnds[i])
            x_ref_train_.append(x_ref_train)
            x_ref_test_.append(x_ref_test)
            y_ref_train_.append(y_ref_train)
            y_ref_test_.append(y_ref_test)
            
        self.x_ref_train,self.x_ref_test,self.y_ref_train,self.y_ref_test = x_ref_train_,x_ref_test_,y_ref_train_,y_ref_test_
        
        
    def create_datasets_shared(self):
        
        # Make target and reference pool distinct
        _,x_tar,_,y_tar = train_test_split(self.x,self.y,test_size=self.target_size,random_state=self.random_state)
        _,x_ref,_,y_ref = train_test_split(self.x,self.y,test_size=self.target_size,random_state=self.random_state+1)
        self.target_data_full = (x_tar,y_tar) # for Yeom et al approach
        self.ref_data_full = (x_ref,y_ref) # for Yeom et al approach
        
        # Split the target data into train and test
        self.x_tar_train,self.x_tar_test,self.y_tar_train,self.y_tar_test = train_test_split(x_tar,y_tar,test_size=0.5,random_state=self.random_state)
        x_ref_train_,x_ref_test_,y_ref_train_,y_ref_test_ = [],[],[],[]
        ref_rnds = [60, 452, 774, 802, 961, 626, 726, 211, 375, 448, 883, 684, 724, 333, 64, 646, 116, 714, 483, 73, 562, 703, 276, 394, 190, 675, 314, 604, 297, 266, 456, 845, 822, 529, 420, 605, 935, 733, 167, 603, 401, 948, 241, 734, 550, 65, 429, 470, 633, 627]
        for i in range(self.n_models):
            x_ref_train,x_ref_test,y_ref_train,y_ref_test = train_test_split(x_ref,y_ref,test_size=0.5,random_state=ref_rnds[i])
            x_ref_train_.append(x_ref_train)
            x_ref_test_.append(x_ref_test)
            y_ref_train_.append(y_ref_train)
            y_ref_test_.append(y_ref_test)
            
        self.x_ref_train,self.x_ref_test,self.y_ref_train,self.y_ref_test = x_ref_train_,x_ref_test_,y_ref_train_,y_ref_test_
        
    def train_target(self,want_tar_tuning,params):
        
        if want_tar_tuning==True:
            self.target_model = RandomForestClassifier(random_state=self.random_state)
            best_params = self.set_parameters(self.target_model,params,self.x_tar_train.iloc[:,:-1],self.y_tar_train.iloc[:,:-1])
            self.target_model.set_params(n_estimators = best_params['n_estimators'])
            self.target_model.set_params(max_depth = best_params['max_depth'])
            
#             print(colored("Best parameters - Target Model",'red'),end=': ')
#             print('n_estimators= {a}, max_depth= {b}'.format(a=best_params['n_estimators'],
#                                                              b=best_params['max_depth']))
            
            self.target_model.fit(self.x_tar_train.iloc[:,:-1],self.y_tar_train.iloc[:,:-1])
        elif want_tar_tuning==False:
            self.target_model = RandomForestClassifier(random_state=self.random_state)
            self.target_model.set_params(n_estimators=self.n_est)
            self.target_model.set_params(max_depth = self.max_d)
            self.target_model.fit(self.x_tar_train.iloc[:,:-1],self.y_tar_train.iloc[:,:-1])
        
    def train_reference(self,want_ref_tuning,params):

        for i in range(self.n_models):
            if want_ref_tuning==True:
                ref_model_ = RandomForestClassifier(random_state=self.random_state)
                best_params = self.set_parameters(ref_model_,params,self.x_ref_train[i].iloc[:,:-1],self.y_ref_train[i].iloc[:,:-1])
                ref_model_.set_params(n_estimators = best_params['n_estimators'])
                ref_model_.set_params(max_depth = best_params['max_depth'])

#                 print(colored("Best parameters - Reference Model",'yellow'),end=': ')
#                 print('n_estimators= {a}, max_depth= {b}'.format(a=best_params['n_estimators'],
#                                                                  b=best_params['max_depth']))

                ref_model_.fit(self.x_ref_train[i].iloc[:,:-1],self.y_ref_train[i].iloc[:,:-1])
            elif want_ref_tuning==False:
                ref_model_ = RandomForestClassifier(random_state=self.random_state)
                ref_model_.set_params(n_estimators=self.n_est)
                ref_model_.set_params(max_depth = self.max_d)
                ref_model_.fit(self.x_ref_train[i].iloc[:,:-1],self.y_ref_train[i].iloc[:,:-1])

            self.ref_model.append(ref_model_)
    
    def target_performance(self):
        return self.target_model.score(self.x_tar_train.iloc[:,:-1],self.y_tar_train.iloc[:,:-1]), self.target_model.score(self.x_tar_test.iloc[:,:-1],self.y_tar_test.iloc[:,:-1])
    
    def get_tar_data(self):
        return self.x_tar_train,self.x_tar_test,self.y_tar_train,self.y_tar_test,self.target_data_full
    
    def get_ref_data(self):
        return self.x_ref_train,self.x_ref_test,self.y_ref_train,self.y_ref_test,self.ref_data_full
    
    def get_models(self):
        return self.ref_model, self.target_model
    
    def get_predprob(self):

        tar_train_pred = self.target_model.predict_proba(self.x_tar_train.iloc[:,:-1])[:,0].reshape(self.x_tar_train.shape[0],1)
        tar_test_pred = self.target_model.predict_proba(self.x_tar_test.iloc[:,:-1])[:,0].reshape(self.x_tar_test.shape[0],1)
#         tar_train_ids = np.array(self.x_tar_train.iloc[:,-1]).reshape(self.x_tar_train.shape[0],1)
#         tar_test_ids = np.array(self.x_tar_test.iloc[:,-1]).reshape(self.x_tar_test.shape[0],1)
        
#         pred_train_target_model = np.concatenate((tar_train_pred,tar_train_ids),axis=-1)
#         pred_test_target_model = np.concatenate((tar_test_pred,tar_test_ids),axis=-1)
#         pred_target_model = np.concatenate((pred_train_target_model,pred_test_target_model))
        pred_target_model = np.concatenate((tar_train_pred,tar_test_pred))
        pred_target = pred_target_model
        
        pred_ref_list = []
        for i in range(self.n_models):
            
            ref_train_pred = self.ref_model[i].predict_proba(self.x_ref_train[i].iloc[:,:-1])[:,0].reshape(self.x_ref_train[i].shape[0],1)
            ref_test_pred = self.ref_model[i].predict_proba(self.x_ref_test[i].iloc[:,:-1])[:,0].reshape(self.x_ref_test[i].shape[0],1)
#             ref_train_ids = np.array(self.x_ref_train[i].iloc[:,-1]).reshape(self.x_ref_train[i].shape[0],1)
#             ref_test_ids = np.array(self.x_ref_test[i].iloc[:,-1]).reshape(self.x_ref_test[i].shape[0],1)
            
#             pred_train_ref_model = np.concatenate((ref_train_pred,ref_train_ids),axis=-1)
#             pred_test_ref_model = np.concatenate((ref_test_pred,ref_test_ids),axis=-1)
            pred_ref_model = np.concatenate((ref_train_pred,ref_test_pred))
            pred_ref_list.append(pred_ref_model)
            
        return pred_target, pred_ref_list
        

In [3]:
def get_SHAP_values(model):
    import shap
    for i in range(points.shape[0]):
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(points[i],check_additivity=False)
        shap_val.append(shap_values[0]) 

def get_dp_impurity(model):
    import numpy as np
    sample_ids =np.arange(0,points.shape[0],1)
    for j in range(len(sample_ids)):
        sp = np.zeros(points.shape[1]+1)
        for i in range(len(model.estimators_)):
            node_indicator = model.estimators_[i].decision_path(points[j,:].reshape(1,points.shape[1]))
            impurity = model.estimators_[i].tree_.impurity
            features = model.estimators_[i].tree_.feature
            node_index = node_indicator.indices[node_indicator.indptr[0]:node_indicator.indptr[1]]
            for node in node_index.tolist():
                f = features[node]
                if f == -2:
                    f=-1
                sp[f] += impurity[node]
        shap_val.append(sp/len(model.estimators_))
        
def get_dp_nodesamples(model):
    import numpy as np
    sample_ids =np.arange(0,points.shape[0],1)
    for j in range(len(sample_ids)):
        sp = np.zeros(points.shape[1]+1)
        for i in range(len(model.estimators_)):
            node_indicator = model.estimators_[i].decision_path(points[j,:].reshape(1,points.shape[1]))
            weighted_n_node_samples = model.estimators_[i].tree_.weighted_n_node_samples
            features = model.estimators_[i].tree_.feature
            node_index = node_indicator.indices[node_indicator.indptr[0]:node_indicator.indptr[1]]
            for node in node_index.tolist():
                f = features[node]
                if f == -2:
                    f=-1
                sp[f] += weighted_n_node_samples[node]
        shap_val.append(sp/len(model.estimators_))
        
def get_dp_importance(model):
    import numpy as np
    sample_ids =np.arange(0,points.shape[0],1)
    for j in range(len(sample_ids)):
        sp = np.zeros(points.shape[1])
        for i in range(len(model.estimators_)):
            node_indicator = model.estimators_[i].decision_path(points[j,:].reshape(1,points.shape[1]))
            importance = model.estimators_[i].feature_importances_
            features = model.estimators_[i].tree_.feature
            node_index = node_indicator.indices[node_indicator.indptr[0]:node_indicator.indptr[1]]
            for node in node_index.tolist():
                f = features[node]
                if f == -2:
                    continue
                sp[f] += importance[f]
        shap_val.append(sp/len(model.estimators_))
        
def get_dp_threshold(model):
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    sample_ids =np.arange(0,points.shape[0],1)
    for j in range(len(sample_ids)):
        sp = np.zeros(points.shape[1]+1)
        for i in range(len(model.estimators_)):
            node_indicator = model.estimators_[i].decision_path(points[j,:].reshape(1,points.shape[1]))
            threshold = MinMaxScaler().fit_transform(model.estimators_[i].tree_.threshold.reshape(-1,1))
            features = model.estimators_[i].tree_.feature
            node_index = node_indicator.indices[node_indicator.indptr[0]:node_indicator.indptr[1]]
            for node in node_index.tolist():
                f = features[node]
                if f == -2:
                    f=-1
                sp[f] += threshold[node]
        shap_val.append(sp)


class attack_dataset():
    
    def __init__(self,rc,tar_model,ref_model,tar_data,ref_data,pred_target,pred_ref,influence):
        
        if influence == 'SHAP':
            self.method = get_SHAP_values
        elif influence == 'DP_impurity':
            self.method = get_dp_impurity
        elif influence == 'DP_threshold':
            self.method = get_dp_threshold
        elif influence == 'DP_importance':
            self.method = get_dp_importance
        elif influence == 'DP_nodesamples':
            self.method = get_dp_nodesamples
        
        res_tar = self.parallelism(rc,tar_data,tar_model)
        res_ref = self.parallelism(rc,ref_data,ref_model)
        self.tsp,self.rsp = self.retrieve((res_tar,res_ref),pred_target,pred_ref)

    def parallelism(self,rc,data_parallel,model):
        view = rc[:]
        view.scatter('points',data_parallel)
        view.scatter('shap_val',[])
        view.block=True
        view.apply(self.method,model)
        res = view.gather('shap_val')
        return res

    def retrieve(self,results_read,pred_target,pred_ref):

        shaps_tar, shaps_ref = results_read[0], results_read[1]
        shaps_tar = np.array(shaps_tar)
        shaps_ref = np.array(shaps_ref)
        tar_shap_prob = np.concatenate((shaps_tar,pred_target),axis=-1)
        ref_shap_prob = np.concatenate((shaps_ref,pred_ref),axis=-1)

        return tar_shap_prob,ref_shap_prob
    
    def get_data(self):
        return self.tsp,self.rsp



In [4]:
class attack():
    
    def __init__(self,r,want_tune,tune_scoring):
        self.rs = r
        self.want_tune=want_tune
        self.tune_scoring=tune_scoring
    
    def set_parameters(self,model,x,y):
        params = {'n_estimators':[20,50,100,150,200],'max_depth':[3,4,5,6,7]}
        grid_search = GridSearchCV(model,params,scoring=self.tune_scoring,n_jobs=-1)
        grid_search.fit(x,y)
        best_parameters = grid_search.best_params_
        return best_parameters

    def fit(self,x,y):
        self.attack_model = RandomForestClassifier(random_state=self.rs)
        if self.want_tune:
            best_params = self.set_parameters(self.attack_model,x,y)
            self.attack_model.set_params(n_estimators = best_params['n_estimators'])
            self.attack_model.set_params(max_depth = best_params['max_depth'])
#             print(colored("Best parameters - Attack model",'blue'),end=": ")
#             print('n_estimators= {a}, max_depth= {b}\n'.format(a=best_params['n_estimators'],b=best_params['max_depth']))

        self.attack_model.fit(x,y)
    
    def predict(self,x):

        return self.attack_model.predict(x)



In [5]:
def baseline(tar_model,target_data,tar_lab,true_lab):
    preds = tar_model.predict(target_data)
    mem_preds = np.ones(preds.shape[0])
    mem_preds[preds!=true_lab]=0
    acc_sc = accuracy_score(tar_lab,mem_preds)
    pr_sc = precision_score(tar_lab,mem_preds)
    rec_sc = recall_score(tar_lab,mem_preds)
    f1_sc =f1_score(tar_lab,mem_preds)

    return acc_sc,pr_sc,rec_sc,f1_sc

In [6]:
'''
DATASET LOAD
'''
def load_data(name):
    
    if name == 'adult_income':

        filepath_text = 'adult-all.txt'
        header_name = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
        data = pd.read_table(filepath_text,sep=',',names=header_name)

        # Encode features
        categorical_feature_mask = data.dtypes==object
        category_col = data.columns[categorical_feature_mask].tolist()
        for col in category_col:
            b, c = np.unique(data[col], return_inverse=True) 
            data[col] = c
        data = data.iloc[:10000,:]
        mm = MinMaxScaler()
        data_trans = mm.fit_transform(data)

        data_trans = pd.DataFrame(data_trans,columns=header_name)
        data_trans['id'] = np.arange(0,data_trans.shape[0])
        y = data_trans[['income','id']]
        x = data_trans.drop(['income'],axis=1)
        
    elif name == 'bcw':

        # Read data
        filepath = '~/bcw/wdbc.data'
        column_name = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
               'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
               'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
               'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
               'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
               'fractal_dimension_se', 'radius_worst', 'texture_worst',
               'perimeter_worst', 'area_worst', 'smoothness_worst',
               'compactness_worst', 'concavity_worst', 'concave points_worst',
               'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32']
        data = pd.read_table(filepath, sep=',', names=column_name)
        data_id = data['id']
        # Create x and y for ML algorithm -- drop unnecesary features (id and last column)
        y = data[['diagnosis','id']]
        x = data.drop(['id','diagnosis','Unnamed: 32'],axis=1)
        x['id'] = data_id
        
    elif name == 'german_credit':

        # Read data
        filepath = '~/german_credit/german.data'
        column_name = ['A'+str(i) for i in range(1,21)]+['risk']
        data = pd.read_table(filepath,sep=' ',names=column_name)
        categorical_feature_mask = data.dtypes==object
        category_col = data.columns[categorical_feature_mask].tolist()
        for col in category_col:
            b, c = np.unique(data[col], return_inverse=True) 
            data[col] = c

        data['id'] = np.arange(0,data.shape[0])
        y = data[['risk','id']]
        x = data.drop(['risk'],axis=1)
        
    elif name == 'pima_diabetes':

        # Read data
        filepath = '~/pima_diabetes/diabetes.csv'
        data = pd.read_csv(filepath)

        data['id'] = np.arange(0,data.shape[0])
        y = data[['Outcome','id']]
        x = data.drop(['Outcome'],axis=1)
        
    elif name == 'hepatitis':

        filepath = '~/hepatitis/hepatitis_clean.csv'
        column_name=['Class','AGE','SEX','STEROID','ANTIVIRALS',
                     'FATIGUE','MALAISE','ANOREXIA','LIVER BIG',
                     'LIVER FIRM','SPLEEN PALPABLE','SPIDERS','ASCITES',
                     'VARICES','BILIRUBIN','ALK PHOSPHATE','SGOT','ALBUMIN',
                     'PROTIME','HISTOLOGY']

        data = pd.read_csv(filepath)
        data['id'] = np.arange(0,data.shape[0])
        y = data[['HISTOLOGY','id']]
        x = data.drop(['HISTOLOGY'],axis=1)
        
    elif name == 'heart_disease':

        filepath = '~/heart_disease/processed.cleveland.data'
        column_name=['age','sex','cp','trestbps','chol','fbs','restecg',
                    'thalach','exang','oldpeak','slope','ca','thal','num']
        data = pd.read_table(filepath,sep=',',names=column_name)

        # 'ca' and 'thal' have '?' in values, hence clean
        data['thal'].replace('?',data['thal'].mode().values[0],inplace=True)
        data['ca'].replace('?',data['ca'].mode().values[0],inplace=True)

        # 'ca' and 'thal' are of type 'object' because the value type is string, hence convert to float
        for col in ['thal','ca']:
            val = data[col]
            val_n = [float(v) for v in val]
            data[col] = pd.Series(val_n)

        data['id'] = np.arange(0,data.shape[0])
        y = data[['num','id']]
        x = data.drop(['num'],axis=1)
        
    return x,y

In [7]:
# write_data

def write_into_files(to_store,run,dataset_name,influence_type,n_est,max_d,metric_store):
    print("\n######### Writing into file #########")
    parent_dir = '/home/amanmoha/influence_attack/results/actual/estimator_analysis'
    while True:
        try:
            directory = dataset_name+'/'+influence_type+'/'+'run'+run+'/'+'n_est-'+str(n_est)
            path = os.path.join(parent_dir,directory)
            os.mkdir(path)
            break
        except FileNotFoundError:
            try: 
                directory = dataset_name+'/'+influence_type+'/'+'run'+run
                path = os.path.join(parent_dir,directory)
                os.mkdir(path)
            except FileNotFoundError:
                try: 
                    directory = dataset_name+'/'+influence_type
                    path = os.path.join(parent_dir,directory)
                    os.mkdir(path)
                    continue
                except FileNotFoundError:
                    directory = dataset_name
                    path = os.path.join(parent_dir,directory)
                    os.mkdir(path)
                    continue
        except FileExistsError:
            break
    
    with open(path+'/data.txt','wb') as f:
        pickle.dump(to_store,f)
    f.close()
    
    with open(path+'/metrics.txt','wb') as f:
        pickle.dump(metric_store,f)
#         f.write("Target train accuracy:\t {a}\nTarget test accuracy:\t {b}\nAttack accuracy:\t {c}\nAttack precision:\t {d}\nAttack recall:\t\t {e}\nAttack f1:\t\t {f}\nBaseline accuracy:\t {g}\nBaseline precision:\t {h}\nBaseline recall:\t {i}\nBaseline f1:\t\t {j}\n".format(a=str(tr_ac),b=str(te_ac),c=str(ac),d=str(ps),e=str(rs),f=str(fs),g=str(ac_b),h=str(ps_b),i=str(rs_b),j=str(fs_b)))
    f.close()
    print("\nWrite complete")

In [8]:
###################
# DRIVER
###################
'''
DATASET NAMES: adult_income, bcw, german_credit, pima_diabetes, hepatitis, heart_disease
KNOWLEGDE: disjoint, shared
INFLUENCE TYPE: SHAP, DP_impurity, DP_threshold, DP_importance, DP_nodesamples
DOUBLE INFLUENCE: False, True
'''
rc = ipp.Client()
## INITIALIZATIONS
dataset_names = ['pima_diabetes']
for dataset_name in tqdm_notebook(dataset_names):
    x,y = load_data(dataset_name)

    ## CONTROL PARAMS
    if dataset_name == 'hepatitis' or dataset_name == 'bcw' or dataset_name == 'german_credit':
        tuning_list = {'n_estimators':[20,50,100],'max_depth':[3,4,5,6]}
        max_d = 4
    elif dataset_name == 'pima_diabetes' or dataset_name == 'heart_disease':
        tuning_list = {'n_estimators':[20],'max_depth':[3]}
        max_d = 3
    elif dataset_name == 'adult_income':
        tuning_list = {'n_estimators':[50,100,200],'max_depth':[4,5,6,7,8]}

    ######### CHANGE HERE
    run = '5'
    #########
    n_states = 50
    n_models = 10
    target_params = {'target_size':0.5,
                    'tuning':False,
                    'scoring':'accuracy',
                    'tune_params':tuning_list}
    attack_params = {'tuning':True,
                    'scoring':'f1'}
    ref_tuning = False

    # Yeom = False
    knowledge = 'disjoint'
    influence_type = 'SHAP'
    n_ests = np.arange(1,31,1)
    write = True
    for n_est in tqdm_notebook(n_ests):

        ac,ps,rs,fs,tr_ac,te_ac=[],[],[],[],[],[]
        ac_b,ps_b,rs_b,fs_b=[],[],[],[]

        #################################
        # print information to screen
        #################################
        print(colored("###################################################",'blue'))
        print(colored("Dataset used: ",'blue'), dataset_name)
        print(colored("Number of samples: ",'blue'), x.shape[0])
        print(colored("Number of features: ",'blue'), x.shape[1])
        print(colored("Number of models: ",'blue'),n_models)
        print(colored("Number of seeds: ",'blue'),n_states)
        print(colored("Number of estimators: ",'blue'),n_est)
        print(colored("Max depth: ",'blue'),max_d)
        print(colored("Number of parallel engines: ",'blue'),len(rc))
        print(colored("Knowledge: ",'blue'), knowledge)
        print(colored("Influence type: ",'blue'),influence_type)
        print(colored("Reference Tuning: ",'blue'),ref_tuning)
        print(colored("###################################################\n",'blue'))

        tar_data_all_states = []
        ref_data_all_states = []
        tar_lab_all_states = []
        ref_lab_all_states = []
        true_labels_all_states = []
        ##### Creating all the models
        for r in tqdm_notebook(rnds[:n_states],desc="Different random states"):
            print("\nRANDOM STATE: ",r)

            ## CREATE MODELS
            all_models = create_models(x,y,target_size=target_params['target_size'],n_models=n_models,random_state=r,
                                       want_tar_tuning=target_params['tuning'],want_ref_tuning=ref_tuning,
                                       tune_scoring=target_params['scoring'], params=target_params['tune_params'],
                                       know=knowledge,n_est=n_est,max_d=max_d)

            ## GET MODELS' DATA
            x_tar_train,x_tar_test,y_tar_train,y_tar_test,target_data_full = all_models.get_tar_data()
            x_ref_train,x_ref_test,y_ref_train,y_ref_test,ref_data_full = all_models.get_ref_data()
            ref_model,target_model = all_models.get_models()
            pred_target,pred_ref = all_models.get_predprob()
            sv1,sv2 = all_models.target_performance()

            true_labels = np.squeeze(np.concatenate((y_tar_train.iloc[:,:-1],y_tar_test.iloc[:,:-1])))
            x_tar_train_red = x_tar_train.iloc[:,:-1]
            x_tar_test_red = x_tar_test.iloc[:,:-1]

            target_data = np.concatenate((x_tar_train_red,x_tar_test_red))
            tar_lab = np.concatenate((np.ones(x_tar_train_red.shape[0]),np.zeros(x_tar_test_red.shape[0])))

            # For reference model

            '''
            Since DP_impurity uses impurities and impurities is also calculated for leaf nodes '-2'.
            '''
            if influence_type == 'DP_impurity' or influence_type == 'DP_threshold' or influence_type == 'DP_nodesamples':
                feats = x.shape[1]+1
            else:
                feats = x.shape[1]

            ref_val = np.zeros((n_models,pred_ref[0].shape[0],feats)) ## changed
            ref_lab = np.zeros((n_models,pred_ref[0].shape[0])) ## changed


            for i in tqdm_notebook(range(n_models),desc="Generating Data"):

                pred_ref_ = pred_ref[i]
                x_ref_train_red = x_ref_train[i].iloc[:,:-1]
                x_ref_test_red = x_ref_test[i].iloc[:,:-1]  ## changed

                ref_data = np.concatenate((x_ref_train_red,x_ref_test_red))
                ref_lab[i] = np.concatenate((np.ones(x_ref_train_red.shape[0]),np.zeros(x_ref_test_red.shape[0])))

                attack_obj = attack_dataset(rc,target_model,ref_model[i],target_data,ref_data,pred_target,pred_ref_,influence_type)
                tar_val,ref_val[i] = attack_obj.get_data()


            ref_val = ref_val.reshape(n_models*(pred_ref[0].shape[0]),feats) ## changed

            ref_lab = ref_lab.reshape(n_models*(pred_ref[0].shape[0])) ## changed
            obj = attack(rnds[0],want_tune=attack_params['tuning'],tune_scoring=attack_params['scoring'])
            obj.fit(ref_val,ref_lab)
            preds = obj.predict(tar_val)

            ## METRICS RECORD
            ac.append(accuracy_score(tar_lab,preds))
            ps.append(precision_score(tar_lab,preds))
            rs.append(recall_score(tar_lab,preds))
            fs.append(f1_score(tar_lab,preds))
            tr_ac.append(sv1)
            te_ac.append(sv2)

            b1,b2,b3,b4=baseline(target_model,target_data,tar_lab,true_labels)
            ac_b.append(b1)
            ps_b.append(b2)
            rs_b.append(b3)
            fs_b.append(b4)

            # to store
            tar_data_all_states.append(tar_val)
            ref_data_all_states.append(ref_val)
            tar_lab_all_states.append(tar_lab)
            ref_lab_all_states.append(ref_lab)
            true_labels_all_states.append(true_labels)

        tar_val = np.array(tar_data_all_states)
        tar_lab = np.array(tar_lab_all_states)
        ref_val = np.array(ref_data_all_states)
        ref_lab = np.array(ref_lab_all_states)
        true_labels = np.array(true_labels_all_states)
        to_store = (tar_val,tar_lab,ref_val,ref_lab,true_labels)
        metric_store = (tr_ac,te_ac,ac,ps,rs,fs,ac_b,ps_b,rs_b,fs_b)
#         print(colored("################# RESULTS #################\n",'blue'))
#         print("Target train accuracy:\t {a}\nTarget test accuracy:\t {b}\nAttack accuracy:\t {c}\nAttack precision:\t {d}\nAttack recall:\t\t {e}\nAttack f1:\t\t {f}\nBaseline accuracy:\t {g}\nBaseline precision:\t {h}\nBaseline recall:\t {i}\nBaseline f1:\t\t {j}\n".format(a=tr_ac/n_states,b=te_ac/n_states,c=ac/n_states,d=ps/n_states,e=rs/n_states,f=fs/n_states,g=ac_b/n_states,h=ps_b/n_states,i=rs_b/n_states,j=fs_b/n_states))
        if write == True:
            write_into_files(to_store,run,dataset_name,influence_type,n_est,max_d,metric_store)

###################################################
Dataset used:  pima_diabetes
Number of samples:  768
Number of features:  9
Number of models:  10
Number of seeds:  50
Number of estimators:  1
Max depth:  3
Number of parallel engines:  20
Knowledge:  disjoint
Influence type:  SHAP
Reference Tuning:  False
###################################################




RANDOM STATE:  60



RANDOM STATE:  452



RANDOM STATE:  774



RANDOM STATE:  802


...


RANDOM STATE:  429



RANDOM STATE:  470



RANDOM STATE:  633



RANDOM STATE:  627



######### Writing into file #########

Write complete

